In [ ]:
!pip install pandas seaborn numpy pyproj geopandas shapely

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import pyproj
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
collision = pd.read_csv('drive/MyDrive/SIAP/processed_data.csv', delimiter=',')
file = open('drive/MyDrive/SIAP/LION_clean.geojson')
road_segment = gpd.read_file(file)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
collision.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-11-24,20:00,MANHATTAN,10006,40.70983,-74.01468,"(40.70983, -74.01468)",WEST STREET,ALBANY STREET,NaN,0.0,0.0,0,0,0,0,0,0,Passing Too Closely,Passing Too Closely,NaN,NaN,NaN,4247215,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2019-11-24,16:11,BROOKLYN,NaN,40.70029,-73.92904,"(40.70029, -73.92904)",CENTRAL AVENUE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,4246795,4 dr sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-11-24,12:20,BROOKLYN,11201,40.69119,-73.99780,"(40.69119, -73.9978)",ATLANTIC AVENUE,HICKS STREET,NaN,1.0,0.0,0,0,0,0,1,0,Unsafe Lane Changing,Unspecified,NaN,NaN,NaN,4246353,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3,2019-09-26,13:07,BROOKLYN,11204,40.63291,-73.97957,"(40.63291, -73.97957)",43 STREET,17 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Backing Unsafely,Unspecified,NaN,NaN,NaN,4215020,Sedan,Sedan,NaN,NaN,NaN
4,2019-11-24,11:51,BRONX,NaN,40.85535,-73.91836,"(40.85535, -73.91836)",MAJOR DEEGAN EXPRESSWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,4246365,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN


In [ ]:
road_segment.head()

,Street,NonPed,StreetWidth_Min,XFrom,YFrom,XTo,YTo,POSTED_SPEED,Snow_Priority,Number_Travel_Lanes,Number_Park_Lanes,Number_Total_Lanes,TRUCK_ROUTE_TYPE,geometry
0,EAST 168 STREET,None,34.0,1010964,241812,1011265,241555,25,S,2,2,4,None,"LINESTRING (-73.90347 40.83036, -73.90238 40.8..."
1,WEST 192 STREET,None,30.0,1011577,255024,1011335,255164,25,S,1,2,3,None,"LINESTRING (-73.90120 40.86662, -73.90207 40.8..."
2,UNION AVENUE,None,34.0,1011601,239640,1011786,240230,25,S,1,2,3,None,"LINESTRING (-73.90118 40.82440, -73.90051 40.8..."
3,UNION AVENUE,None,34.0,1011601,239640,1011786,240230,25,S,1,2,3,None,"LINESTRING (-73.90118 40.82440, -73.90051 40.8..."
4,UNION AVENUE,None,34.0,1011601,239640,1011786,240230,25,S,1,2,3,None,"LINESTRING (-73.90118 40.82440, -73.90051 40.8..."


In [ ]:
road_segment['Street'] = road_segment['Street'].str.strip()
collision['ON STREET NAME'] = collision['ON STREET NAME'].str.strip()

In [ ]:
collision = collision[collision['ON STREET NAME'].isin(road_segment['Street'].unique())]
collision.shape

(754269, 29)

In [ ]:
#import time
collision['geometry'] = np.nan

def near(point, data):
    #start = time.time()
    rs = data.geometry.unary_union
    if rs:
      nearest = data.geometry.distance(nearest_points(point, rs)[1]) < 1e-6
      #print(time.time() - start)
      return data[nearest].geometry.iloc[0]

#start = time.time()
for index, row in collision.iterrows():
    collision.loc[index, 'geometry'] = near(Point(float(row.LONGITUDE), float(row.LATITUDE)), road_segment.loc[road_segment['Street'] == row['ON STREET NAME']])
    print(index)

#collision['geometry'] = collision.apply(lambda row: near(Point(float(row.LONGITUDE), float(row.LATITUDE)), road_segment.loc[road_segment['Street'] == row['ON STREET NAME']]
#), axis=1

Streaming output truncated to the last 5000 lines.
308338
308339
308340
308341
308343
308344
308345
308347
308348
308349
308350
308351
308352
308353
308354
308355
308356
308359
308360
308361
308364
308365
308366
308367
308368
308369
308370
308371
308373
308374
308375
308376
308378
308380
308382
308383
308385
308386
308387
308388
308389
308390
308391
308392
308393
308394
308396
308397
308399
308400
308402
308404
308406
308408
308410
308413
308415
308417
308418
308419
308420
308421
308422
308423
308424
308427
308429
308430
308431
308432
308433
308434
308435
308437
308438
308440
308441
308442
308444
308445
308446
308447
308448
308449
308450
308451
308452
308454
308455
308460
308461
308462
308463
308464
308466
308468
308469
308470
308472
308473
308474
308475
308476
308481
308482
308483
308484
308485
308487
308489
308493
308494
308497
308498
308499
308500
308501
308502
308509
308510
308511
308512
308515
308516
308517
308518
308519
308521
308522
308525
308526
308532
308533
308534
308535
3085

In [ ]:
collision.head()

In [ ]:
collision['geometry'] = collision['geometry'].astype(str)
road_segment['geometry'] = road_segment['geometry'].astype(str)

In [ ]:
collision = pd.merge(collision, road_segment, on='geometry')

In [ ]:
collision.head()

In [ ]:
collision.to_csv('data.csv', index=False)

In [ ]:
!cp data.csv "drive/MyDrive/"